In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time

In [26]:
def get_percentile(x, t='latest',val=0):
    if t=='latest':
        x=x.dropna().values
        val=x[-1]
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    else:
        return  np.percentile(np.sort(x),val)

In [12]:
#holdings=['GDX','RHHBY','ETSY','ECL','OLLI','BLK','SPY','SPXL','AZN','CDNS','CWH','BYND']
holdings=['ZS']

In [33]:
holdings='CMG,SPY,DOCU,ZS,FB,OLLI,SHOP,MBUU,ADBE,BL,QQQ,MDB,BLK,CDNS,MELI,AZN,NOW,NYT,ECL,CWH,BYND,DDOG,BIG,TREX,TW,DKNG'


In [34]:
holdings=holdings.split(',')

In [35]:
res=[[],[],[],[],[],[]]

In [ ]:
c=1
for ticker in holdings:
    c+=1
    if c==5:
        c=1
        time.sleep(60)
    d={'function': 'TIME_SERIES_DAILY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )
    spy=[[],[],[],[],[],[],[]]
    for key, val in response.json()['Time Series (Daily)'].items():
        spy[0].append(float(val['1. open']))
        spy[1].append(float(val['2. high']))
        spy[2].append(float(val['3. low']))
        spy[3].append(float(val['4. close']))
        spy[4].append(float(val['5. adjusted close']))
        #spy[5].append(float(val['6. volume']))
        #spy[6].append(float(val['7. dividend amount']))
    t=list(response.json()['Time Series (Daily)'].keys())
    try:
        if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
            print('Notlive')
            d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'False','chartLast':'1'}
            response = requests.get(
            'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format(ticker)#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
            ,params=d,
            )
            
            spy[0].insert(0,float(response.json()[0]['open']))
            spy[1].insert(0,float(response.json()[0]['high']))
            spy[2].insert(0,float(response.json()[0]['low']))
            spy[3].insert(0,float(response.json()[0]['close']))
            spy[4].insert(0,float(response.json()[0]['close']))
            t.insert(0,str(response.json()[0]['date']))

            #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
    except:
        print(ticker,'hi')


        
        
    spy=pd.DataFrame({'Date':t,
                 'open':spy[0],
                  'high':spy[1],
                  'low':spy[2],
                  'close':spy[3],
                  'adj_close':spy[4]#,
                #  'volume':spy[5],
                #  'dividend':spy[6],
                 })
    spy=spy.astype({'Date': 'datetime64'})
    spy=spy.sort_values(by='Date')
    spy['highest_high']=spy.high.rolling(22).max()
    spy['lowest_low']=spy.low.rolling(22).min()
    spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
    spy['k']=spy.k.rolling(3).mean()
    spy['k_change'] = spy.k/spy.k.shift(3)-1
    res[0].append(ticker)
    res[1].append(spy.iloc[-1]['Date']) 
    res[2].append(spy.iloc[-1]['k']) 
    res[3].append(spy.iloc[-1]['k_change'])
    res[4].append(np.nanpercentile(spy.k_change,q=10))
    res[5].append(get_percentile(spy.k_change))


Notlive
Notlive
Notlive
Notlive
Notlive
MBUU hi
Notlive
Notlive
BL hi
Notlive
Notlive
Notlive
CDNS hi
Notlive
MELI hi
Notlive
BYND hi
Notlive


In [ ]:
res=pd.DataFrame({'ticker':res[0], 
            'date':res[1],
             'k':res[2],
             'k_change':res[3],
             'k_percentile':res[4],
             'k_change_percentile':res[5]
             }
            )

In [ ]:
res.sort_values(by='k_change',ascending=True)

In [11]:
res.sort_values(by='k',ascending=True)

,ticker,date,k,k_change,k_percentile
2,ZS,2020-05-22,0.842161,0.122192,-0.432043
3,ZS,2020-05-26,0.849068,0.108612,-0.431713
1,DOCU,2020-05-26,0.885129,0.030342,-0.387830
0,SPY,2020-05-26,0.912545,0.014094,-0.454851


In [24]:
res[(res.k>0.7) & (res.k_change>0)].sort_values(by='k',ascending=False)

,ticker,date,k,k_change,k_percentile
5,OLLI,2020-05-26,0.986562,0.054041,-0.413098
25,BIG,2020-05-26,0.971424,0.020082,-0.484475
27,TREX,2020-05-26,0.959997,0.025107,-0.478566
22,CWH,2020-05-26,0.946102,0.020497,-0.516107
20,NYT,2020-05-26,0.934016,0.161639,-0.486296
11,MDB,2020-05-26,0.933195,0.022307,-0.413329
24,DDOG,2020-05-26,0.930085,0.012230,-0.250558
9,BL,2020-05-26,0.908847,0.042083,-0.380678
28,TW,2020-05-26,0.873882,0.037369,-0.413804
29,DKNG,2020-05-26,0.872189,0.011878,-0.288886


In [ ]:
pk_444bcc6ef4a94b859405ef30fe66fa14 
https://cloud.iexapis.com/stable/stock/QQQ/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=1

In [170]:
if ~(pd.to_datetime(list(response.json()['Time Series (Daily)'].keys())[0])==pd.Timestamp.today()):
    d={'function': 'TIME_SERIES_INTRADAY','apikey':"YEUC2DL0F7KC43PB",'symbol':'MBUU','interval':'1min'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )

In [189]:
d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'True','chartLast':'1'}
response2 = requests.get(
        'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format('MBUU')#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
        ,params=d,
        )

In [180]:
t=pd.DataFrame(response.json()[0]['date'],response.json()[0]['open'],response.json()[0]['high'],response.json()[0]['low'],response.json()[0]['close'],response.json()[0]['close'])

TypeError: __init__() takes from 1 to 6 positional arguments but 7 were given

In [190]:
date=list(response.json()['Time Series (Daily)'].keys())

In [192]:
date.insert(0,response2.json()[0]['date'])

In [193]:
date

['2020-05-20',
 '2020-05-19',
 '2020-05-18',
 '2020-05-15',
 '2020-05-14',
 '2020-05-13',
 '2020-05-12',
 '2020-05-11',
 '2020-05-08',
 '2020-05-07',
 '2020-05-06',
 '2020-05-05',
 '2020-05-04',
 '2020-05-01',
 '2020-04-30',
 '2020-04-29',
 '2020-04-28',
 '2020-04-27',
 '2020-04-24',
 '2020-04-23',
 '2020-04-22',
 '2020-04-21',
 '2020-04-20',
 '2020-04-17',
 '2020-04-16',
 '2020-04-15',
 '2020-04-14',
 '2020-04-13',
 '2020-04-09',
 '2020-04-08',
 '2020-04-07',
 '2020-04-06',
 '2020-04-03',
 '2020-04-02',
 '2020-04-01',
 '2020-03-31',
 '2020-03-30',
 '2020-03-27',
 '2020-03-26',
 '2020-03-25',
 '2020-03-24',
 '2020-03-23',
 '2020-03-20',
 '2020-03-19',
 '2020-03-18',
 '2020-03-17',
 '2020-03-16',
 '2020-03-13',
 '2020-03-12',
 '2020-03-11',
 '2020-03-10',
 '2020-03-09',
 '2020-03-06',
 '2020-03-05',
 '2020-03-04',
 '2020-03-03',
 '2020-03-02',
 '2020-02-28',
 '2020-02-27',
 '2020-02-26',
 '2020-02-25',
 '2020-02-24',
 '2020-02-21',
 '2020-02-20',
 '2020-02-19',
 '2020-02-18',
 '2020-02-

In [211]:
spy


,Date,open,high,low,close,adj_close,highest_high,lowest_low,k,k_change
5032,2000-05-18,89.00,89.36,85.38,85.75,75.0926,NaN,NaN,NaN,NaN
5031,2000-05-19,83.75,85.00,81.31,81.69,71.5372,NaN,NaN,NaN,NaN
5030,2000-05-22,81.63,82.25,75.58,81.63,71.4846,NaN,NaN,NaN,NaN
5029,2000-05-23,81.38,82.63,74.25,74.81,65.5123,NaN,NaN,NaN,NaN
5028,2000-05-24,75.00,79.88,72.25,79.50,69.6194,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4,2020-05-14,218.12,221.87,215.99,221.83,221.8300,228.095,203.63,0.727483,-0.253806
3,2020-05-15,218.77,223.33,218.33,223.27,223.2700,228.095,203.63,0.729614,-0.205561
2,2020-05-18,226.28,228.68,225.47,227.43,227.4300,228.680,203.63,0.832266,0.039379
1,2020-05-19,227.66,229.85,226.71,226.86,226.8600,229.850,203.63,0.879615,0.209120


In [93]:
np.nanpercentile(spy.k_change,q=10)

-0.45913701443037197

In [174]:
response.json()[0]

{'date': '2020-05-20',
 'minute': '12:30',
 'label': '12:30 PM',
 'high': None,
 'low': None,
 'open': None,
 'close': None,
 'average': None,
 'volume': 0,
 'notional': 0,
 'numberOfTrades': 0}

In [198]:
pd.to_datetime('today')

Timestamp('2020-05-20 12:48:48.505275')

In [200]:
pd.Timestamp.today().date()

datetime.date(2020, 5, 20)

In [201]:
pd.to_datetime(t[0]).date()

datetime.date(2020, 5, 19)

In [203]:
pd.to_datetime(t[0]).date()==pd.Timestamp.today().date()

False